In [1]:
# Clear all memory
from IPython import get_ipython
get_ipython().magic('reset -f')

from transformers import pipeline
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.prompts import StringPromptTemplate
from langchain.chains import LLMChain
from langchain.tools import BaseTool
from langchain.llms import HuggingFacePipeline
import requests

# Initialize the Phi-4 model pipeline
pipe = pipeline("text-generation", model="meta-llama/Llama-3.3-70B-Instruct", trust_remote_code=True)

/tmp/ipykernel_16725/1323800296.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -f')
/home/ubuntu/marbles/Projects/miniconda/envs/optimat-llm-313/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

In [9]:
# Create custom tool for navigation API calls
class NavigationTool(BaseTool):
    name: str = "navigation"
    description: str = "Validates if origin and destination addresses are valid locations"
    
    def _run(self, origin: str, destination: str):
        response = requests.post(
            'http://127.0.0.1:5000/api/navigate',
            json={"origin": origin, "destination": destination},
            headers={"Content-Type": "application/json"}
        )
        return response.json()

    def _arun(self, origin: str, destination: str):
        raise NotImplementedError("Async not supported")

# Create prompt template for the agent
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    input_variables: list[str]

    def format(self, **kwargs) -> str:
        return self.template.format(**kwargs)

template = """You are a helpful transportation assistant. When a user asks about travel between two locations:
1. ALWAYS extract the origin and destination
2. Use the navigation tool with the EXACT locations mentioned
3. Provide a helpful response based on the tool's output

Example:
Question: "I want to go from Seattle to Portland"
Thought: I need to validate these locations
Action: navigation
Action Input: {"origin": "Seattle", "destination": "Portland"}

Question: {input}
Thought: """
# Initialize tools and agent
nav_tool = NavigationTool()
tools = [nav_tool]

# Create LangChain wrapper for Phi-4
llm = HuggingFacePipeline(pipeline=pipe)

from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
import re

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> AgentAction | AgentFinish:
        # Check if this is a final answer
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Extract action and action input
        action_match = re.search(r'Action: (.*?)[\n]', llm_output, re.DOTALL)
        action_input_match = re.search(r'Action Input: (.*)', llm_output, re.DOTALL)
        
        if not action_match or not action_input_match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
            
        action = action_match.group(1).strip()
        action_input = action_input_match.group(1).strip()
        
        return AgentAction(tool=action, tool_input=action_input, log=llm_output)

# Create the agent
prompt = CustomPromptTemplate(
    template=template,
    input_variables=["input"]
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
output_parser = CustomOutputParser()

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    tools=tools,
    stop=["Observation:", "Final Answer:"]
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Function to handle a single chat interaction
def handle_chat_input(user_input):
    try:
        response = agent_executor.invoke({"input": user_input})
        return f"Assistant: {response['output']}"
    except Exception as e:
        return f"Error: {str(e)}"

# Example usage in notebook:
# response = handle_chat_input("I want to go from New York to Los Angeles")
# print(response)

In [10]:
response = handle_chat_input("I want to go from New York to Los Angeles")
print(response)



> Entering new AgentExecutor chain...
Error: '"origin"'
